# Calculate syllable features

Optional notebook within the chronic ephys processing pipeline
- 1-preprocess_acoustics
- 2-curate_acoustics
- 3-sort_spikes
- 4-curate_spikes
- 5-cluster_acoustics
    - **5.1-syllable_feature_analysis**

Use the environment **birdsong** to run this notebook

In [1]:
%matplotlib inline

import os
import sys
import pickle
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from scipy import stats
from scipy import signal
from scipy.io import wavfile
import IPython.display as ipd
from scipy.stats import pearsonr
from joblib import Parallel, delayed
import noisereduce as nr
from sklearn.model_selection import train_test_split

sys.path.append('/mnt/cube/lo/envs/ceciestunepipe')
from ceciestunepipe.file import bcistructure as et

sys.path.append('/mnt/cube/lo/envs/SAT')
from calculate_SAT_features import calculate_features

## Set parameters

In [2]:
# session parameters
sess_par = {
    'bird':'z_c5o30_23', # bird ID
    'sess':'2023-06-15', # session date
    'ephys_software':'sglx', # recording software, sglx or oe
    'stim_sess':False, # if song stimulus was played during the session, ignore detected bouts
    'trim_bouts':True, # manually trim bouts after curation
    'sort':'sort_0', # sort index
}

## Load segmented syllables

In [3]:
sess_epochs = et.list_ephys_epochs(sess_par)
print(f"Found {len(sess_epochs)} epoch(s):", sess_epochs)

Found 1 epoch(s): ['0913_g0']


In [4]:
this_epoch = sess_epochs[0] # set epoch index
epoch_struct = et.sgl_struct(sess_par,this_epoch,ephys_software=sess_par['ephys_software'])
print('Loading syllables from epoch', this_epoch)

# load segmented syllable dataframe
syl_seg_df_path = os.path.join(epoch_struct['folders']['derived'],'syl_seg_df_ap0.pickle')
with open(syl_seg_df_path, 'rb') as handle:
    syl_seg_df = pickle.load(handle)

Loading syllables from epoch 0913_g0


## Calculate syllable features

In [5]:
features_in = syl_seg_df.columns.tolist()
syl_seg_df = calculate_features(syl_seg_df)
features_out = syl_seg_df.columns.tolist()
print('Added the following syllable features:',set(features_out) - set(features_in))

Added the following syllable features: {'FM', 'goodness', 'AM', 'f0', 'duration_ms', 'mean_freq', 'amplitude', 'entropy'}


In [6]:
syl_seg_df.head(3)

,file,sess,epoch,sample_rate,bout_index,waveform_raw,waveform_filt,start_ms_ap_0,start_sample_ap_0,syllable_index,...,label_orig,label,duration_ms,f0,mean_freq,amplitude,goodness,FM,AM,entropy
0,/mnt/cube/chronic_ephys/der/z_c5o30_23/2023-06...,2023-06-15,0913_g0,40000,0,"[14, 23, 19, 16, -2, 37, 51, 49, 42, 20, 37, 3...","[3.3851349619418847, 13.522975407927271, 20.71...",919480,27584486,0,...,4,0,56,"[525.3561668435311, 540.7502775664981, 543.882...","[2479.6318876837963, 2479.5869596466646, 2470....","[143.27679854240318, 144.39858516668593, 145.3...","[0.10259739092556909, 0.14397099678767095, 0.1...","[0.7879531835604661, 0.7879531835604661, 0.787...","[2584544.6849159594, 3156681.601859254, 361846...","[-1.891032298654375, -1.991935816617417, -2.06..."
1,/mnt/cube/chronic_ephys/der/z_c5o30_23/2023-06...,2023-06-15,0913_g0,40000,0,"[14, 23, 19, 16, -2, 37, 51, 49, 42, 20, 37, 3...","[3.3851349619418847, 13.522975407927271, 20.71...",919480,27584486,1,...,4,0,68,"[493.0141887514742, 537.6357851471714, 553.389...","[2832.560229112974, 2899.5090834358793, 2938.4...","[144.49206284367574, 146.26663331306088, 147.8...","[0.20561146103683567, 0.20486051174835548, 0.1...","[0.7879531835604661, 0.7879531835604661, 0.787...","[4496961.870308619, 5987643.710685434, 8064415...","[-1.7842421751760806, -1.868247029859285, -1.7..."
2,/mnt/cube/chronic_ephys/der/z_c5o30_23/2023-06...,2023-06-15,0913_g0,40000,0,"[14, 23, 19, 16, -2, 37, 51, 49, 42, 20, 37, 3...","[3.3851349619418847, 13.522975407927271, 20.71...",919480,27584486,2,...,0,0,76,"[nan, 550.2024259228027, 556.5954701647858, 55...","[2755.020615371183, 2846.292909025462, 2877.64...","[144.2056433642707, 146.12090270642864, 147.92...","[0.111279747049674, 0.14580015887779169, 0.148...","[0.7879531835604661, 0.7879531835604661, 0.787...","[4914220.5674833665, 7137293.607432798, 975089...","[-1.8719154057561873, -2.0816475598860418, -2...."


## Save syllable features

In [7]:
syl_seg_df.to_pickle(os.path.join(epoch_struct['folders']['derived'],'syl_seg_df_ap0.pickle'))